In [1]:
import pandas as pd
import glob
import os
import datetime
import numpy as np

In [2]:
def recrusive_file_gen(root_path):
    for root,dirs,files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [3]:
historical_daily=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
historical_daily_df=pd.DataFrame({"file_path":historical_daily})
historical_daily_df['date']=historical_daily_df['file_path'].apply(lambda x: x.split("vel/MediaStormDailySales_week_ending_")[1][:10])
historical_daily_df=historical_daily_df[historical_daily_df['date']>="2017-02-05"]
print(historical_daily_df.shape)
print(historical_daily_df['date'].max())

(70, 2)
2018-06-09


In [4]:
daily_2018=list(recrusive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
daily_2018=[x for x in daily_2018 if ("aily" in x) & (".txt" in x)]

daily_2018_df=pd.DataFrame({"file_path":daily_2018})

daily_2018_df['date']=daily_2018_df['file_path'].apply(lambda x: x.split("2018_by_weeks/MediaStorm_")[1][:10])

daily_2018_df=daily_2018_df[daily_2018_df['date']>"2018-06-09"]

print(daily_2018_df.shape)
print(daily_2018_df['date'].max())


(29, 2)
2018-12-29


In [5]:
daily_2019=list(recrusive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_2019=[x for x in daily_2019 if ("aily" in x) & (".txt" in x)]

daily_2019_df=pd.DataFrame({"file_path":daily_2019})

daily_2019_df['date']=daily_2019_df['file_path'].apply(lambda x: x.split("2019_by_weeks/MediaStorm_")[1][:10])

daily_2019_df=daily_2019_df[daily_2019_df['date']<="2019-02-02"]

print(daily_2019_df.shape)
print(daily_2019_df['date'].max())

(5, 2)
2019-02-02


In [6]:
all_file=historical_daily_df['file_path'].tolist()+daily_2018_df['file_path'].tolist()+daily_2019_df['file_path'].tolist()
len(all_file)


104

In [9]:
i_counter=0
df_rewards_summary=pd.DataFrame()
for file in all_file:
    df=pd.read_csv(file,sep="|",dtype=str,usecols=['location_id','transaction_dt','customer_id_hashed','subclass_transaction_amt'])
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df=df[df['location_id']!="6990"]
    del df['location_id']
    df_date_max=df['transaction_dt'].max()
    df_date_min=df['transaction_dt'].min()
    
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    del df['transaction_dt']
    del df['customer_id_hashed']
    # Sales only
    df=df.groupby(['rewards_label'])['subclass_transaction_amt'].sum().to_frame().reset_index()
    
    df['week_start']=df_date_min
    df['week_end']=df_date_max
    
    df_rewards_summary=df_rewards_summary.append(df)
    print(datetime.datetime.now(),i_counter)
    
    i_counter+=1
    
    if i_counter%10==1:
        print(datetime.datetime.now(),i_counter)
        
    

2019-04-30 21:44:35.087113 0
2019-04-30 21:44:35.087733 1
2019-04-30 21:45:05.531883 1
2019-04-30 21:45:27.522751 2
2019-04-30 21:45:49.346422 3
2019-04-30 21:46:14.148736 4
2019-04-30 21:46:35.826028 5
2019-04-30 21:48:02.974756 6
2019-04-30 21:48:57.871506 7
2019-04-30 21:50:38.695453 8
2019-04-30 21:51:02.313541 9
2019-04-30 21:51:46.647000 10
2019-04-30 21:51:46.647554 11
2019-04-30 21:53:19.955063 11
2019-04-30 21:54:13.118526 12
2019-04-30 21:57:46.143607 13
2019-04-30 21:59:10.210961 14
2019-04-30 22:01:46.155430 15
2019-04-30 22:02:30.143238 16
2019-04-30 22:06:16.164097 17
2019-04-30 22:06:38.105922 18
2019-04-30 22:07:41.094893 19
2019-04-30 22:08:39.675186 20
2019-04-30 22:08:39.675699 21
2019-04-30 22:10:22.579631 21
2019-04-30 22:11:30.805384 22
2019-04-30 22:14:07.052012 23
2019-04-30 22:16:10.410664 24
2019-04-30 22:16:50.813826 25
2019-04-30 22:17:39.713993 26
2019-04-30 22:18:59.518955 27
2019-04-30 22:19:56.485787 28
2019-04-30 22:22:01.069203 29
2019-04-30 22:24:26.9

In [10]:
df_rewards_summary.shape

(208, 4)

In [11]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Jian_Checking_Friend_and_Family_Dates'

In [12]:
df_rewards_summary.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Jian_Checking_Friend_and_Family_Dates/rewards_sales_in_MMM_JL_0430.csv",index=False)

In [ ]:
df_rewards_summary['rewards_ratio']=df_rewards_summary

In [14]:
df_rewards_summary.head(6)

,rewards_label,subclass_transaction_amt,week_start,week_end
0,Non_Rewards,3.173442e+07,2017-08-20,2017-08-26
1,Rewards,4.645362e+07,2017-08-20,2017-08-26
0,Non_Rewards,3.499755e+07,2017-10-22,2017-10-28
1,Rewards,4.809804e+07,2017-10-22,2017-10-28
0,Non_Rewards,3.298405e+07,2017-10-15,2017-10-21
1,Rewards,4.714253e+07,2017-10-15,2017-10-21


In [29]:
df_rewards_summary_wide=df_rewards_summary.pivot(index='week_start',columns='rewards_label',values='subclass_transaction_amt').reset_index()
df_rewards_summary_wide=df_rewards_summary_wide.sort_values('week_start')
df_rewards_summary_wide['total']=df_rewards_summary_wide['Rewards']+df_rewards_summary_wide['Non_Rewards']
df_rewards_summary_wide['rewards_ratio']=df_rewards_summary_wide['Rewards']/df_rewards_summary_wide['total']


In [31]:
df_rewards_summary_wide.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Jian_Checking_Friend_and_Family_Dates/wide_rewards_sales_in_MMM_JL_0430.csv",index=False)

In [26]:
4.458004/(4.458004+3.005504)

0.597306789247094